<a href="https://colab.research.google.com/github/villafue/Capstone_1-_Predict_House_Prices/blob/master/Reference%20Notebooks/Final_Table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Machine Learning Algorithm (MLA) Selection and Initialization
dt = DecisionTreeRegressor(max_depth=3, max_features='log2')
models = [LinearRegression(),
          Ridge(alpha=0.03727272727272728, normalize=True, random_state=29), 
          Lasso(alpha=0.0001, random_state=29), 
          ElasticNet(alpha=0.001, l1_ratio=0.1, random_state=29), 
          RandomForestRegressor(n_estimators=200, random_state=29), 
          AdaBoostRegressor(base_estimator=dt, n_estimators=200, learning_rate=0.20, random_state=29), 
          GradientBoostingRegressor(alpha=0.7, learning_rate=0.08, max_depth=1, min_samples_split=25, 
                                    min_samples_leaf=10, n_estimators=2000, random_state=29),
          xgb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1, colsample_bynode=1, 
                           colsample_bytree=0.45, gamma=0.03, importance_type='gain', learning_rate=0.01, 
                           max_delta_step=0, max_depth=3, max_samples=0.2, min_child_weight=1, missing=None, 
                           n_estimators=2000, n_jobs=1, nthread=None, objective='reg:squarederror', random_state=29, 
                           reg_alpha=0.08, reg_lambda=0.04, scale_pos_weight=1, seed=None, silent=None, 
                           subsample=0.12)]

# First I will use ShuffleSplit as a way of randomising the cross validation samples.
shuff = ShuffleSplit(n_splits=5, test_size=0.2, random_state=29)

#create table to compare MLA metrics
columns = ['Name', 'Parameters', 'Train Accuracy Mean', 'Test Accuracy']
final_model_compare = pd.DataFrame(columns = columns)

#index through models and save performance to table
row_index = 0
for alg in models:

    #set name and parameters
    model_name = alg.__class__.__name__
    final_model_compare.loc[row_index, 'Name'] = model_name
    final_model_compare.loc[row_index, 'Parameters'] = str(alg.get_params())
    
    alg.fit(X_train, Y_train)
    
    #score model with cross validation: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate
    training_results = np.sqrt((-cross_val_score(alg, X_train, Y_train, cv = shuff, scoring= 'neg_mean_squared_error')).mean())
    test_results = np.sqrt(((Y_test-alg.predict(X_test))**2).mean())
    
    final_model_compare.loc[row_index, 'Train Accuracy Mean'] = (training_results)
    final_model_compare.loc[row_index, 'Test Accuracy'] = (test_results)
    
    row_index+=1
    print(row_index, alg.__class__.__name__, 'trained...')

decimals = 4
final_model_compare['Train Accuracy Mean'] = final_model_compare['Train Accuracy Mean'].apply(lambda x: round(x, decimals))
final_model_compare['Test Accuracy'] = final_model_compare['Test Accuracy'].apply(lambda x: round(x, decimals))
print('\n', '-' * 136)
final_model_compare
print('\n', '-' * 136)

1 LinearRegression trained...
2 Ridge trained...
3 Lasso trained...
4 ElasticNet trained...
5 RandomForestRegressor trained...
6 AdaBoostRegressor trained...
7 GradientBoostingRegressor trained...
8 XGBRegressor trained...

 ----------------------------------------------------------------------------------------------------------------------------------------


,Name,Parameters,Train Accuracy Mean,Test Accuracy
0,LinearRegression,"{'copy_X': True, 'fit_intercept': True, 'n_job...",0.1121,0.1194
1,Ridge,"{'alpha': 0.03727272727272728, 'copy_X': True,...",0.1117,0.1201
2,Lasso,"{'alpha': 0.0001, 'copy_X': True, 'fit_interce...",0.1118,0.1200
3,ElasticNet,"{'alpha': 0.001, 'copy_X': True, 'fit_intercep...",0.1118,0.1215
4,RandomForestRegressor,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri...",0.1353,0.1462
5,AdaBoostRegressor,"{'base_estimator__ccp_alpha': 0.0, 'base_estim...",0.1607,0.1703
6,GradientBoostingRegressor,"{'alpha': 0.7, 'ccp_alpha': 0.0, 'criterion': ...",0.1145,0.1261
7,XGBRegressor,"{'base_score': 0.5, 'booster': 'gbtree', 'cols...",0.1122,0.1189


In [ ]:
dr = DummyRegressor()
dr.fit(X_train, Y_train)
dr.score(X_test, Y_test)
training_results = np.sqrt((-cross_val_score(dr, X_train, Y_train, cv = 5, scoring= 'neg_mean_squared_error')).mean())
test_results = np.sqrt(((Y_test-dr.predict(X_test))**2).mean())
print('The training RMSE is: {:.4f}'.format(training_results))
print('The testing RMSE is: {:.4f}'.format(test_results))

The training RMSE is: 0.3859
The testing RMSE is: 0.4188


In [ ]:
# Import StackingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV

dt = DecisionTreeRegressor(max_depth=3, max_features='log2')
estimators = [('lr', LinearRegression()),
              ('ridge', Ridge(alpha=0.03727272727272728, normalize=True, random_state=29)),
              ('lasso', Lasso(alpha=0.0001, random_state=29)),
              ('eln', ElasticNet(alpha=0.001, l1_ratio=0.1, random_state=29)),
              ('rf', RandomForestRegressor(n_estimators=200, random_state=29)),
              ('ada', AdaBoostRegressor(base_estimator=dt, n_estimators=200, learning_rate=0.20,
                                        random_state=29)),
              ('gbr', GradientBoostingRegressor(alpha=0.7, learning_rate=0.08, max_depth=1, 
                                                min_samples_split=25, min_samples_leaf=10, 
                                                n_estimators=2000, random_state=29)),
              ('xgb', xgb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1, 
                                       colsample_bynode=1, colsample_bytree=0.45, gamma=0.03, 
                                       importance_type='gain', learning_rate=0.01, max_delta_step=0, 
                                       max_depth=3, max_samples=0.2, min_child_weight=1, missing=None, 
                                       n_estimators=2000, n_jobs=1, nthread=None, objective='reg:squarederror', 
                                       random_state=29, reg_alpha=0.08, reg_lambda=0.04, scale_pos_weight=1, 
                                       seed=None, silent=None, subsample=0.12)) 
]

# Instantiate StackingRegressor
reg5 = StackingRegressor(estimators=estimators, verbose=2, passthrough=True, n_jobs=-1)

reg5.fit(X_train, Y_train)

training_results = np.sqrt((-cross_val_score(reg5, X_train, Y_train, cv = 5, scoring= 'neg_mean_squared_error')).mean())
test_results = np.sqrt(((Y_test-reg5.predict(X_test))**2).mean())
print('\n', '-' * 136)
print('The training RMSE is: {:.4f}'.format(training_results))
print('The testing RMSE is: {:.4f}'.format(test_results))


 ----------------------------------------------------------------------------------------------------------------------------------------
The training RMSE is: 0.1127
The testing RMSE is: 0.1164


In [1]:
import pandas as pd

In [17]:
fin_tabl = {'Model': ['Dummy Regressor', 'Linear Regression', 'XGBoost', 'Stacking Regressor' ],
            'TrainRMSE': [0.3859, 0.1121, 0.1122, 0.1127],
            'TestRMSE': [0.4188, 0.1194, 0.1189, 0.1164],
            '%_Inc_Last_Mod': ['N/A', '71.5%', '0.4%', '2.1%'],
            '%_Inc_Base_Mod': ['N/A', '71.5%', '71.6%', '72.2%']
            }
fin_tabl = pd.DataFrame(data=fin_tabl)
fin_tabl

,Model,TrainRMSE,TestRMSE,%_Inc_Last_Mod,%_Inc_Base_Mod
0,Dummy Regressor,0.3859,0.4188,N/A,N/A
1,Linear Regression,0.1121,0.1194,71.5%,71.5%
2,XGBoost,0.1122,0.1189,0.4%,71.6%
3,Stacking Regressor,0.1127,0.1164,2.1%,72.2%


In [11]:
1 - .1164/.4188

0.7220630372492837

In [10]:
1 - .1189/.4188

0.7160936007640879

In [6]:
1 - .1194/.4188

0.7148997134670487

In [ ]:
1 - .1189/.1194

0.0041876046901172526

In [ ]:
1 - .1164/.1189

0.021026072329688783

In [ ]:
1 - .1164/.1180

0.013559322033898202

In [ ]:
1 - .1164/.4188

0.7220630372492837